# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 10, 25, 20, 15

while True:
    dt = datetime.now()
    if dt.month >= month and dt.day >= day:
        if dt.hour > hour or (dt.hour == hour and dt.minute > minute):
            break
    sleep(60)

# Parameter optimization

In [2]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '4h'
opt_limit = 300
load = False

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'RSI': {'timeperiod': [12, 14, 16], 'low_bound': [15, 20, 25, 30, 35]},
              'STOCH': {'fastk_period': [5, 7, 9, 11], 'slowk_period': [2, 3, 4, 5],
                        'slowd_period': [3, 5, 7, 9], 'low_bound': [10, 15, 20, 25]}}
# optim_dict = {'RSI': {'timeperiod': [16], 'low_bound': [30]},
#                   'STOCH': {'fastk_period': [5], 'slowk_period': [4],
#                             'slowd_period': [2], 'low_bound': [30]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['indicator_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat = stat.sort_values(['indicator_rank', 'pct_right_forecast_avg'], ascending=False)
stat.head(20)

Number of combinations is 3840


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3840/3840 [1:38:41<00:00,  1.54s/it]


,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,indicator_rank
389,STOCH_RSI,12,20,9,2,5,15,100.00,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,-0.56,-0.82,-1.04,-1.32,-1.53,-1.78,-1.86,-1.68,-1.76,-1.77,-2.09,-2.10,-2.19,-2.30,-2.50,-2.37,-2.35,-2.28,-2.03,-2.00,-2.30,-2.47,-2.24,-1.99,9,89.352917,174.176250
390,STOCH_RSI,12,20,9,2,5,20,100.00,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,-0.56,-0.82,-1.04,-1.32,-1.53,-1.78,-1.86,-1.68,-1.76,-1.77,-2.09,-2.10,-2.19,-2.30,-2.50,-2.37,-2.35,-2.28,-2.03,-2.00,-2.30,-2.47,-2.24,-1.99,9,89.352917,174.176250
453,STOCH_RSI,12,20,11,2,5,15,100.00,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,-0.58,-0.85,-1.34,-1.32,-1.53,-1.78,-1.86,-1.68,-1.76,-1.77,-2.09,-2.10,-2.19,-2.30,-2.50,-2.37,-2.43,-2.28,-2.03,-2.00,-2.30,-2.47,-2.45,-1.99,9,89.352917,174.176250
454,STOCH_RSI,12,20,11,2,5,20,100.00,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,88.89,-0.58,-0.85,-1.34,-1.32,-1.53,-1.78,-1.86,-1.68,-1.76,-1.77,-2.09,-2.10,-2.19,-2.30,-2.50,-2.37,-2.43,-2.28,-2.03,-2.00,-2.30,-2.47,-2.45,-1.99,9,89.352917,174.176250
483,STOCH_RSI,12,20,11,4,3,25,85.71,78.57,85.71,78.57,71.43,78.57,78.57,78.57,78.57,85.71,78.57,78.57,85.71,85.71,78.57,78.57,78.57,85.71,85.71,85.71,92.86,92.86,85.71,85.71,-0.83,-0.91,-1.35,-1.35,-1.36,-1.38,-1.42,-1.47,-1.20,-1.38,-1.48,-2.01,-1.97,-2.10,-2.30,-2.35,-2.35,-2.30,-2.13,-2.20,-2.25,-2.53,-2.46,-2.00,14,82.438333,174.136667
3106,STOCH_RSI,16,25,5,4,3,20,85.71,92.86,92.86,92.86,85.71,85.71,85.71,85.71,85.71,85.71,78.57,78.57,78.57,78.57,78.57,78.57,78.57,78.57,78.57,78.57,78.57,78.57,71.43,71.43,-0.57,-0.59,-0.93,-1.26,-1.21,-1.44,-1.36,-1.42,-1.56,-1.65,-1.58,-1.64,-1.74,-1.87,-1.89,-1.71,-1.75,-1.66,-1.59,-1.64,-1.86,-1.57,-1.65,-1.60,14,81.843750,165.812500
3297,STOCH_RSI,16,25,11,4,3,15,83.33,91.67,91.67,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,75.00,-0.49,-0.58,-0.80,-1.26,-1.07,-1.20,-1.33,-1.41,-1.40,-1.42,-1.49,-1.83,-1.88,-2.02,-2.09,-2.35,-2.35,-2.27,-2.02,-2.00,-2.17,-2.19,-2.11,-1.85,12,83.677917,164.135000
3269,STOCH_RSI,16,25,11,2,5,15,91.67,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,83.33,75.00,-0.49,-0.58,-1.00,-1.24,-1.06,-0.95,-1.30,-1.31,-1.35,-1.21,-1.33,-1.90,-1.97,-1.94,-2.09,-1.97,-2.35,-2.27,-1.97,-1.98,-2.17,-2.19,-1.96,-1.85,12,83.330417,159.965000
3525,STOCH_RSI,16,30,11,2,5,15,94.44,88.89,88.89,83.33,83.33,77.78,83.33,83.33,83.33,77.78,77.78,77.78,77.78,77.78,72.22,72.22,72.22,72.22,72.22,72.22,72.22,77

# Save new config data to config file

In [3]:
from config_updater import ConfigUpdater

timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'RSI': {'timeperiod': [12], 'low_bound': [20]},
                  'STOCH': {'fastk_period': [9], 'slowk_period': [2],
                            'slowd_period': [5], 'low_bound': [15]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)